In [1]:
import pandas as pd
import json
import sqlite3
import os

***1. Load Transational Data***

In [2]:
orders = pd.read_csv('orders.csv')
print(f"Orders Loaded. Shape:{orders.shape}")

Orders Loaded. Shape:(10000, 6)


***2. Load User Master Data***

In [3]:
with open('users.json', 'r') as f:
    users_data = json.load(f)
users = pd.DataFrame(users_data)
print(f"Users Loaded. Shape: {users.shape}")

Users Loaded. Shape: (3000, 4)


***3. Load Restaurant Master Data***

In [6]:
# Create connection to a temporary database

conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Read and execute the SQL script

with open('restaurants.sql', 'r') as f:
    sql_script = f.read()
cursor.executescript(sql_script)

# Read the table into a DataFrame (assuming the table name is 'restaurants')

restaurants = pd.read_sql_query("SELECT * FROM restaurants", conn)
print(f"Restaurants Loaded. Shape: {restaurants.shape}")

Restaurants Loaded. Shape: (500, 4)


In [10]:
# Join 1: Merge Orders with Users on 'user_id'

merged_data = pd.merge(orders, users, on = 'user_id', how = 'left')

# Join 2: Merge the result with Restaurants on 'restaurant_id'

final_df = pd.merge(merged_data, restaurants, on = 'restaurant_id', how = 'left')

# Save the final version as requested

final_df.to_csv('final_food_delivery_dataset.csv', index = False)
print("Final dataset created: final_food_delivery_dataset.csv")

Final dataset created: final_food_delivery_dataset.csv


In [11]:
final_df.isnull().sum()

order_id             0
user_id              0
restaurant_id        0
order_date           0
total_amount         0
restaurant_name_x    0
name                 0
city                 0
membership           0
restaurant_name_y    0
cuisine              0
rating               0
dtype: int64

In [12]:
final_df.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [13]:
final_df.tail()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
9995,9996,2528,249,21-05-2023,1211.96,Royal Kitchen North Indian,User_2528,Hyderabad,Gold,Restaurant_249,Italian,4.7
9996,9997,2867,267,06-08-2023,1188.05,Darbar Cafe Punjabi,User_2867,Bangalore,Regular,Restaurant_267,Chinese,4.2
9997,9998,522,420,11-11-2023,979.44,Ruchi Tiffins Chinese,User_522,Bangalore,Gold,Restaurant_420,Italian,4.0
9998,9999,319,492,08-09-2023,1105.93,Swagath Kitchen North Indian,User_319,Bangalore,Gold,Restaurant_492,Italian,4.0
9999,10000,457,439,21-10-2023,879.58,Classic Cafe Family Restaurant,User_457,Chennai,Regular,Restaurant_439,Mexican,3.1


**Solutions for MCQ Questions:**

***1. Which city has the highest total revenue (total_amount) from Gold members?***

In [22]:
gold_members = final_df[final_df['membership'] == 'Gold']
res1 = gold_members.groupby('city')['total_amount'].sum().idxmax()
print(f"Answer: {res1}")

Answer: Chennai


***2. Which cuisine has the highest average order value across all orders?***

In [23]:
res2 = final_df.groupby('cuisine')['total_amount'].mean().idxmax()
print(f"Answer: {res2}")

Answer: Mexican


***3. How many distinct users placed orders worth more than ₹1000 in total (sum of all their orders)?***

In [24]:
user_totals = final_df.groupby('user_id')['total_amount'].sum()
count = len(user_totals[user_totals > 1000])
print(f"Total Users: {count}")

Total Users: 2544


***4. Which restaurant rating range generated the highest total revenue?***

In [26]:
# Create bins for the rating ranges
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ['3.0 - 3.5', '3.6 - 4.0', '4.1 - 4.5', '4.6 - 5.0']

# Ensure 'rating' is numeric before cutting
final_df['rating'] = pd.to_numeric(final_df['rating'], errors='coerce')

# Create the range column
final_df['rating_range'] = pd.cut(final_df['rating'], bins=bins, labels=labels)

# Add observed=False to clear the warning
res4 = final_df.groupby('rating_range', observed=False)['total_amount'].sum().idxmax()

print(f"Answer: {res4}")

Answer: 4.6 - 5.0


***5. Among Gold members, which city has the highest average order value?***

In [32]:
gold_members = final_df[final_df['membership'] == 'Gold']
city_aov = gold_members.groupby('city')['total_amount'].mean()
print(f"City with highest Gold AOV: {city_aov.idxmax()} (Value: {city_aov.max():.2f})")

City with highest Gold AOV: Chennai (Value: 808.46)


***6. Which cuisine has the lowest number of distinct restaurants but still contributes significant revenue?***

In [67]:
summary = final_df.groupby("cuisine").agg(
    restaurants=("restaurant_id","nunique"),
    revenue=("total_amount","sum")
)
summary.sort_values(["restaurants","revenue"], ascending=[True,False])

,restaurants,revenue
cuisine,,
Chinese,120,1930504.65
Italian,126,2024203.80
Indian,126,1971412.58
Mexican,128,2085503.09


***7. What percentage of total orders were placed by Gold members? (Rounded to nearest integer)***

In [38]:
# Calculate exact percentage
gold_pct = (len(gold_members) / len(final_df)) * 100

# Print both values
print(f"Exact Percentage: {gold_pct:.2f}%")
print(f"Rounded Answer: {round(gold_pct)}%")

Exact Percentage: 49.87%
Rounded Answer: 50%


***8. Which restaurant has the highest average order value but less than 20 total orders?***

In [70]:
stats = final_df.groupby("restaurant_name_x").agg(
    avg_order=("total_amount","mean"),
    orders=("order_id","count")
)

stats[stats.orders < 20].sort_values("avg_order", ascending=False).head(1)

,avg_order,orders
restaurant_name_x,,
Hotel Dhaba Multicuisine,1040.222308,13


****- The Correct Answer is Hotel Dhaba Multicuisine****

***Q8. The correct answer among the given options***

In [51]:
# 1. Define the options provided in the question
options = [
    'Grand Cafe Punjabi', 
    'Grand Restaurant South Indian', 
    'Ruchi Mess Multicuisine', 
    'Ruchi Foods Chinese'
]

# 2. Filter data for these specific restaurants
filtered_df = final_df[final_df['restaurant_name_x'].isin(options)]

# 3. Calculate Average Order Value and Order Count
stats = filtered_df.groupby('restaurant_name_x').agg(
    avg_order_value=('total_amount', 'mean'),
    order_count=('order_id', 'count')
)

# 4. Apply the condition: Less than 20 total orders
eligible_restaurants = stats[stats['order_count'] < 20]

# 5. Get the final answer (Highest AOV)
ans_restaurant = eligible_restaurants['avg_order_value'].idxmax()
ans_value = eligible_restaurants['avg_order_value'].max()

print(f"--- Among the given options, the correct answer is ---")
print(f"Restaurant: {ans_restaurant}")
print(f"Average Order Value: {ans_value:.2f}")

--- Among the given options, the correct answer is ---
Restaurant: Ruchi Foods Chinese
Average Order Value: 686.60


***9. Which combination contributes the highest revenue?***

In [54]:
# 1. Group by membership and cuisine and sum the total revenue
revenue_combinations = final_df.groupby(['membership', 'cuisine'])['total_amount'].sum()

# 2. Check the specific options mentioned in your question
options = [
    ('Gold', 'Indian'),
    ('Gold', 'Italian'),
    ('Regular', 'Indian'),
    ('Regular', 'Chinese')
]

print("--- REVENUE BY COMBINATION ---")
for membership, cuisine in options:
    # Use .get() to avoid errors if a combination is missing
    rev = revenue_combinations.get((membership, cuisine), 0)
    print(f"{membership} + {cuisine}: ₹{rev:,.2f}")

# 3. Print the winner
winner = revenue_combinations.idxmax()
print(f"\nWINNER (Highest Overall): {winner[0]} + {winner[1]}")

--- REVENUE BY COMBINATION ---
Gold + Indian: ₹979,312.31
Gold + Italian: ₹1,005,779.05
Regular + Indian: ₹992,100.27
Regular + Chinese: ₹952,790.91

WINNER (Highest Overall): Regular + Mexican


***10. During which quarter of the year is the total revenue highest?***

In [57]:
# Fixed code with dayfirst=True to remove the warning
final_df['order_date'] = pd.to_datetime(final_df['order_date'], dayfirst=True)

# Calculate the quarter
final_df['quarter'] = final_df['order_date'].dt.to_period('Q')

# Find the highest revenue quarter
ans10 = final_df.groupby('quarter')['total_amount'].sum().idxmax()

print(f"Highest Revenue Quarter: {ans10}")

Highest Revenue Quarter: 2023Q3


***This section contains questions with numerical answers. Please enter the correct values, including decimals wherever applicable***

***11. How many total orders were placed by users with Gold membership?***

In [58]:
gold_orders_count = len(final_df[final_df['membership'] == 'Gold'])
print(f"Total Gold Orders: {gold_orders_count}")

Total Gold Orders: 4987


***Q12.***

In [59]:
hyd_revenue = final_df[final_df['city'] == 'Hyderabad']['total_amount'].sum()
print(f"Hyderabad Total Revenue: {round(hyd_revenue)}")

Hyderabad Total Revenue: 1889367


***Q13.***

In [61]:
distinct_users = final_df['user_id'].nunique()
print(f"Distinct Users Count: {distinct_users}")

Distinct Users Count: 2883


***Q14.***

In [77]:
round(final_df[final_df.membership=="Gold"]["total_amount"].mean(), 2)

np.float64(797.15)

***Q15.***

In [79]:
(final_df.rating >= 4.5).sum()

np.int64(3374)

***Q16.***

In [80]:
top_city = final_df[final_df.membership=="Gold"] \
            .groupby("city")["total_amount"].sum().idxmax()

final_df[(final_df.membership=="Gold") & (final_df.city==top_city)].shape[0]

1337